## Classification (PA3-1,2,3)

In [1]:
## Data generation

In [2]:
from basic_fcn import *
from utils import *
import torch.optim as optim
import time
from torch.utils.data import DataLoader
import torch
import gc
import copy
from dataloader import *
from starter import *


#hyper param
epochs = 30   
lr = 0.0001
criterion = nn.CrossEntropyLoss()# Choose an appropriate loss function from https://pytorch.org/docs/stable/_modules/torch/nn/modules/loss.html
n_class = 10

print("creating model...")
fcn_model = FCN(n_class=n_class)
fcn_model.apply(init_weights)

# gpu
gpu_status = torch.cuda.is_available()
print("GPU:", gpu_status)

if gpu_status : 
    device = torch.device('cuda') # determine which device to use (gpu or cpu)
else : 
    device = torch.device('cpu')

fcn_model.to(device)
    
print("Training...")
val(fcn_model, 0)  # show the accuracy before training
model_name="latest_model_3"
train_loss_record, valid_loss_record = train(fcn_model, epochs,lr, save_fp=model_name) 

KeyboardInterrupt: 

In [ ]:
#plot
plt.plot(np.arange(epochs), train_record, label= "Training Loss")
plt.plot(np.arange(epochs), valid_record, label="Validation Loss")
plt.xlabel("Epoches")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [1]:
#test image
test()

model_name = 'latest_model_3'
visualize(model_name)


NameError: name 'visualize' is not defined